In [0]:
# Carga de los secretos
container_name = dbutils.secrets.get(scope="secretliga1", key="adlscontenedor")
adls_account_name = dbutils.secrets.get(scope="secretliga1", key="storageaccountidt")
client_id = dbutils.secrets.get(scope="secretliga1", key="clientid")
client_secret = dbutils.secrets.get(scope="secretliga1", key="secretidt")
tenant_id = dbutils.secrets.get(scope="secretliga1", key="tenantidt")

# Configuración de Spark para la autenticación
spark.conf.set(f"fs.azure.account.auth.type.{adls_account_name}.dfs.core.windows.net", "OAuth")
spark.conf.set(f"fs.azure.account.oauth.provider.type.{adls_account_name}.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set(f"fs.azure.account.oauth2.client.id.{adls_account_name}.dfs.core.windows.net", client_id)
spark.conf.set(f"fs.azure.account.oauth2.client.secret.{adls_account_name}.dfs.core.windows.net", client_secret)
spark.conf.set(f"fs.azure.account.oauth2.client.endpoint.{adls_account_name}.dfs.core.windows.net", f"https://login.microsoftonline.com/{tenant_id}/oauth2/v2.0/token")

# Comando para montar el ADLS
configs = {"fs.azure.account.auth.type": "OAuth",
           "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
            "fs.azure.account.oauth2.client.id": client_id,
            "fs.azure.account.oauth2.client.secret": client_secret,
            "fs.azure.account.oauth2.client.endpoint": f"https://login.microsoftonline.com/{tenant_id}/oauth2/v2.0/token"}

try:
  dbutils.fs.mount(
         source = f"abfss://{container_name}@{adls_account_name}.dfs.core.windows.net/",
          mount_point = "/mnt/liga1",
          extra_configs = configs
     )
  print("ADLS montado exitosamente.")
except Exception as e:
  if "already mounted" in str(e):
    print("ADLS ya está montado.")
  else:
    raise e

# Muestra el contenido del punto de montaje para verificar
display(dbutils.fs.ls("/mnt/liga1"))